In [73]:
// The idea was inspired by a series of videos from "The Coding Train" channel
// https://www.youtube.com/watch?v=9zfeTw-uFCw&list=PLRqwX-V7Uu6bJM3VgzjNV5YxVxUwzALHV

function chooseByProbability(
	values: string[],
	probabilities: number[],
): string {
	const randomValue = Math.random();
	let cumulative = 0;
	for (let i = 0; i < values.length; i++) {
		cumulative += probabilities[i];
		if (randomValue < cumulative) {
			return values[i];
		}
	}
	return values[values.length - 1];
}

function crossover(value1: string, value2: string): string {
	const pivot = Math.floor(Math.random() * value1.length);
	return value1.slice(0, pivot) + value2.slice(pivot);
}

function mutate(value: string, probability = 0.01): string {
	return value
		.split("")
		.map((char) => {
			if (Math.random() < probability) {
				return String.fromCharCode(Math.floor(Math.random() * 256));
			}
			return char;
		})
		.join("");
}

// Currently Deno doesn't support sharing varible types between cells 
// in notebook, so the workaround is to declare types globally
// https://github.com/denoland/vscode_deno/issues/932
declare global {
	interface GenerationStats {
		averageFitness: number;
		minFitness: number;
		maxFitness: number;
		bestFitValue: string;
	}
	
	class Generation {
		public readonly values: string[];
		public readonly size: number;
		public readonly targetLength: number;
		static createRandom(size: number, targetLength: number): Generation;
		createNextGeneration(target: string): Generation;
		getStats(target: string): GenerationStats;
	}
}

export class Generation {
	constructor(
		public readonly values: string[],
		public readonly size: number,
		public readonly targetLength: number,
	) {}

	static createRandom(size: number, targetLength: number): Generation {
		const values = Array.from(
			{ length: size },
			() =>
				Array.from(
					{ length: targetLength },
					() => String.fromCharCode(Math.floor(Math.random() * 256)),
				).join(""),
		);
		return new Generation(values, size, targetLength);
	}

	private calculateFitness(target: string): number[] {
		return this.values.map((value) => {
			let fitness = 0;
			for (let i = 0; i < value.length; i++) {
				if (target[i] === value[i]) {
					fitness += 1;
				}
			}
			return Math.pow(fitness, 3);
		});
	}

	createNextGeneration(target: string): Generation {
		const fitnesses = this.calculateFitness(target);
		const totalFitness = fitnesses.reduce((a, b) => a + b, 0);
		const probabilities = fitnesses.map((fitness) => fitness / totalFitness);

		const values = Array.from({ length: this.size }, () => {
			const value1 = chooseByProbability(this.values, probabilities);
			const value2 = chooseByProbability(this.values, probabilities);
			return mutate(crossover(value1, value2));
		});
		return new Generation(values, this.size, this.targetLength);
	}

	getStats(target: string): GenerationStats {
		const fitnesses = this.calculateFitness(target);
		const maxFitness = Math.max(...fitnesses);
		return {
			averageFitness: fitnesses.reduce((a, b) => a + b, 0) / this.size,
			minFitness: Math.min(...fitnesses),
			maxFitness,
			bestFitValue: this.values[fitnesses.indexOf(maxFitness)],
		};
	}
}


In [74]:
// The goal is to evolve a population of random strings over successive
// generations to match a given target string.
// For our target we gonna use the famous line from William Shakespeare's play "Hamlet."
const target = "To be or not to be, that is the question"

let generation = Generation.createRandom(target.length * 50, target.length);
let currentGeneration = 0;
const stats: GenerationStats[] = [];

console.time();
while (true) {
	generation = generation.createNextGeneration(target);
	currentGeneration++;

	const generationStats = generation.getStats(target);
	console.log(
		`Generation ${currentGeneration}: ${generationStats.bestFitValue}`,
	);
	stats.push(generationStats);

	if (generation.values.some((value) => value === target)) {
		break;
	}
}
console.timeEnd();

declare global {
	const stats: GenerationStats[];
}

Generation 1: ToP/-¥x Ú·tÈ^fescÕ}Iø¬iH²c qP¼¾CG/
Generation 2: To,/-¥x Ú·_îÕ]£ÖDxlßhÞtðOþd qP¼¾CG/
XßIø¬iH²c qPsÉI~x Ú·tÈ%oj
Generation 4: ToP#ox Ú·tÈ^fescÕ}hÞtðOþd qP¼¾CG/
Generation 5: To =#o­|KÈ¡Ê ´Õ}Itå6¡tî qPsÉI~
Generation 6: ­ =#ox Ú·tÈ^o?ÖT,É«`²êi1MtO q=s?Ë$¨
Generation 7: To¯ ox ÚÖtÈ^o?ÖT,É«`²êi1MtO q=s?Ë$¨
Generation 8: To¯ o Ú·tÈ¼o?ømJ}hÞtÕå6Mtc qßµ¶tv7±
/ÙZÜtJi1ÛtO q=stv7±Úqtêo?
Generation 10: To¯ ox ÚÖtÈ^o?ÖT,É«`²êi1MtO qßµ¶tvoê
Generation 11: To ¯ o Ú·tÈÖo?ømJ}hÞti1¡tîe qu»sÉL~
Generation 12: To ¯ o Ú·tÈÖo?íømJ}hÞti1¡tîe qPsÉoT
Generation 13: To = o Ú·tÈÖo?ømJ}hÞti1¡tîe qPsÉHoT
Generation 14: To = o |ot Úo?ÁÔßhÞtià/e qßµ>tÃod
Generation 15: To = o Ú·tÈ¼o?gecÕ}Þt 0æ tîe qPs¾iã÷
Generation 16: To =¯ o Ú·tÈ^ï ´lßhËt iû(¤ quMstoT
Generation 17: To ¯ o Ú·tã^f?gç`hÞtÈi1¡tîe qu»stvo×
Generation 18: Tob/ o éotÈ^o?ÚT,lhÞt 0æ a¸ quMs¾ið÷
Generation 19: Tob/ o dotÈ^o?Öe!lßhÞtêi1MtOe 

In [75]:
import { document } from "jsr:@ry/jupyter-helper";
import * as Plot from "npm:@observablehq/plot";

Plot.plot({
  grid: true,
  color: {
    legend: true,
    domain: ["max", "avg", "min"],
    range: ["#10b981", "#efb118", "#f43f5e"],
  },
  marks: [
    Plot.ruleY([0]),
    Plot.lineY(
      stats.map(({ averageFitness }, i) => ({
        generation: i,
        averageFitness,
        name: "avg",
      })),
      { stroke: "name", x: "generation", y: "averageFitness" },
    ),
    Plot.lineY(
      stats.map(({ minFitness }, i) => ({
        generation: i,
        minFitness,
        name: "min",
      })),
      { stroke: "name", x: "generation", y: "minFitness" },
    ),
    Plot.lineY(
      stats.map(({ maxFitness }, i) => ({
        generation: i,
        maxFitness,
        name: "max",
      })),
      { stroke: "name", x: "generation", y: "maxFitness" },
    ),
  ],
  y: { label: "Fitness" },
  x: { label: "Generation" },
  style: {
    background: "transparent",
  },
  document,
});


max avg min 0 5,000 10,000 15,000 20,000 25,000 30,000 35,000 40,000 45,000 50,000 55,000 60,000 ↑ Fitness 0 50 100 150 200 250 300 Generation → <path d="M40,369.951L41.914,369.777L43.828,369.475L45.743,369.101L47.657,368.71L49.571,368.29L51.485,367.759L53.399,367.056L55.314,366.238L57.228,365.221L59.142,364.152L61.056,362.89L62.97,361.698L64.884,360.575L66.799,359.326L68.713,357.942L70.627,356.567L72.541,355.161L74.455,353.801L76.37,351.826L78.284,350.279L80.198,348.413L82.112,346.547L84.026,344.549L85.941,342.367L87.855,340.475L89.769,338.375L91.683,336.215L93.597,334.259L95.512,331.722L97.426,329.325L99.34,326.439L101.254,323.698L103.168,320.814L105.083,318.004L106.997,314.864L108.911,311.855L110.825,309.157L112.739,306.459L114.653,303.98L116.568,301.798L118.482,299.734L120.396,297.517L122.31,295.246L124.224,292.208L126.139,290.374L128.053,288.552L129.967,285.938L131.881,283.742L133.795,281.035L135.71,279.61L137.624,277.789L139.538,276.834L141.452,275.167L143.366,273.591L145.281,272.212L147.195,270.955L149.109,269.8L151.023,268.619L152.937,266.82L154.851,265.376L156.766,264.252L158.68,263.407L160.594,262.374L162.508,261.03L164.422,260.368L166.337,258.736L168.251,257.126L170.165,256.25L172.079,255.279L173.993,255.405L175.908,254.643L177.822,254.612L179.736,252.781L181.65,251.77L183.564,250.386L185.479,249.708L187.393,249.262L189.307,248.048L191.221,246.317L193.135,244.749L195.05,244.007L196.964,243.252L198.878,242.115L200.792,242.355L202.706,241.508L204.62,240.519L206.535,239.347L208.449,239.234L210.363,238.516L212.277,237.146L214.191,235.846L216.106,234.096L218.02,232.571L219.934,232.047L221.848,230.071L223.762,229.755L225.677,229.504L227.591,228.93L229.505,227.178L231.419,226.16L233.333,225.81L235.248,224.297L237.162,223.949L239.076,222.868L240.99,221.342L242.904,220.838L244.818,220.529L246.733,219.614L248.647,219.583L250.561,218.464L252.475,218.378L254.389,217.298L256.304,215.855L258.218,214.43L260.132,214.538L262.046,214.654L263.96,213.27L265.875,212.385L267.789,212.34L269.703,212.19L271.617,212.753L273.531,212.11L275.446,211.289L277.36,212.158L279.274,211.903L281.188,211.682L283.102,211.929L285.017,211.785L286.931,211.964L288.845,211.368L290.759,211.542L292.673,211.017L294.587,210.62L296.502,210.08L298.416,209.154L300.33,208.114L302.244,207.564L304.158,206.258L306.073,206.623L307.987,206.04L309.901,205.01L311.815,205.776L313.729,206.425L315.644,205.205L317.558,205.063L319.472,204.811L321.386,206.254L323.3,205.588L325.215,204.696L327.129,205.284L329.043,204.606L330.957,203.436L332.871,203.622L334.785,202.414L336.7,201.615L338.614,202.866L340.528,202.274L342.442,203.191L344.356,202.998L346.271,202.661L348.185,202.608L350.099,202.355L352.013,202.232L353.927,202.691L355.842,203.223L357.756,202.488L359.67,202.624L361.584,202.105L363.498,203.007L365.413,201.682L367.327,200.668L369.241,201.299L371.155,200.501L373.069,199.98L374.983,198.678L376.898,200.209L378.812,200.619L380.726,199.955L382.64,198.901L384.554,197.3L386.469,197.023L388.383,196.909L390.297,197.228L392.211,197.755L394.125,197.009L396.04,197.333L397.954,197.493L399.868,195.423L401.782,194.285L403.696,193.217L405.611,192.615L407.525,190.909L409.439,190.674L411.353,191.205L413.267,190.852L415.182,189.2L417.096,188.102L419.01,186.908L420.924,186.63L422.838,186.673L424.752,186.931L426.667,188.025L428.581,188.116L430.495,188.042L432.409,189.345L434.323,188.974L436.238,186.595L438.152,187.097L440.066,186.247L441.98,185.803L443.894,185.755L445.809,185.471L447.723,184.868L449.637,184.466L451.551,183.87L453.465,182.91L455.38,181.748L457.294,182.301L459.208,181.12L461.122,180.753L463.036,179.798L464.95,179.582L466.865,178.629L468.779,178.01L470.693,175.904L472.607,176.326L474.521,176.346L476.436,176.341L478.35,176.364L480.264,175.376L482.178,174.652L484.092,174.744L486.007,173.889L487.921,173.706L489.835,174.054L491.749,173.062L493.663,172.756L495.578,171.933L497.492,172.111L499.406,171.46L501.32,171.603L503.234,171.089L505.149,169.966L507.063,168.171L508.977,169.081L510.89